# Esercizio 5 - Topic Modelling

L'obiettivo è trovare i topic principali di un corpus di testi.  
Come corpus utilizzo i testi di 4 pagine di Wikipedia relative agli stessi 4 argomenti trattati nell'esercizio 4:
- Lebanon
- Racing bicycle
- Labrador retriever
- Indie rock

Questi testi li unisco in un unica lista che, dopo la fase di preprocessing, sarà il corpus su cui allenare il modello scelto.  
Il modello scelto è **LDA** (Latent Dirichlet Allocation), fornito dalla libreria *Gensim*.

In [3]:
import wikipediaapi
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import LdaModel

## Creo i testi di riferimento

Scarico da Wikipedia i testi delle 4 pagine e li unisco in una lista. Ogni elemento della lista è una stringa che rappresenta il testo di una pagina Wikipedia.

In [9]:
# Try wiki api
wiki_wiki = wikipediaapi.Wikipedia('en')

lebanon_page = wiki_wiki.page('Lebanon')
bike_page = wiki_wiki.page('Racing bicycle')
labrador_page = wiki_wiki.page('Labrador retriever')
indie_page = wiki_wiki.page('Indie rock')

lebanon_text = lebanon_page.text
bike_text = bike_page.text
labrador_text = labrador_page.text
indie_text = indie_page.text

docs = [lebanon_text, bike_text, labrador_text, indie_text]

## Preprocessing dei documenti

Tramite la libreria *nltk* eseguo il preprocessing dei documenti.  
Dopodichè con la funzione `gensim.corpora.Dictionary()` creo un dizionario che associa ad ogni parola un indice.  
Da questo dizionario rimuovo le parole che appaiono in più del 50% dei documenti.  
Infine costruisco una lista di documenti, dove ogni documento è rappresentato come una bag of words.

In [37]:
# Tokenize, remove stopwords, lemmatize a list of documents
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocess(docs):
    prep = docs.copy()
    for i in range(len(docs)):
        prep[i] = prep[i].lower()
        prep[i] = tokenizer.tokenize(prep[i])
        prep[i] = [word for word in prep[i] if word not in stop_words]
        prep[i] = [lemmatizer.lemmatize(word) for word in prep[i]]
        # Remove single characters
        prep[i] = [word for word in prep[i] if len(word) > 1]
        # Remove word if is a number
        prep[i] = [word for word in prep[i] if not word.isnumeric()]
    return prep

preprocessed_docs = preprocess(docs)

# Create dictionary
dictionary = Dictionary(preprocessed_docs)

# Filter out words that occur more than 50% of the documents 
# -- no_below=1 means that we keep words that appear at least in one document --> meaning all words in the dictionary (because we have only 4 documents)
dictionary.filter_extremes(no_below=1, no_above=0.5)

# Create corpus as a bag of words representation of the documents
corpus = [dictionary.doc2bow(doc) for doc in preprocessed_docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 3792
Number of documents: 4


## Alleno il modello LDA

Con la funzione `gensim.models.ldamodel.LdaModel()` alleno il modello LDA con i documenti preprocessati.  
Il modello viene allenato con 4 topic per vedere se riesce a trovare gli argomenti principali dei documenti.  
Giocando con *passes* e *iterations* si può migliorare la qualità del modello:
- *passes*: numero di passaggi per sul corpus
- *iterations*: numero di iterazioni per ogni documento

In [35]:
# Train LDA model
n_topics = 4
epochs = 40
iterations = 3000

tmp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

lda_model = LdaModel(corpus=corpus, id2word=dictionary.id2token, num_topics=n_topics, passes=epochs, iterations=iterations)

lda_model.print_topics()

[(0,
  '0.027*"labrador" + 0.025*"dog" + 0.011*"retriever" + 0.010*"breed" + 0.006*"newfoundland" + 0.005*"black" + 0.005*"coat" + 0.005*"first" + 0.005*"bred" + 0.004*"earl"'),
 (1,
  '0.026*"indie" + 0.025*"rock" + 0.024*"band" + 0.012*"music" + 0.009*"pop" + 0.009*"scene" + 0.008*"term" + 0.008*"punk" + 0.007*"mainstream" + 0.007*"like"'),
 (2,
  '0.040*"lebanon" + 0.017*"lebanese" + 0.006*"war" + 0.006*"christian" + 0.005*"government" + 0.005*"beirut" + 0.005*"arab" + 0.005*"muslim" + 0.004*"mount" + 0.004*"al"'),
 (3,
  '0.028*"bicycle" + 0.019*"racing" + 0.014*"road" + 0.010*"frame" + 0.010*"wheel" + 0.010*"bike" + 0.009*"cm" + 0.008*"tire" + 0.006*"must" + 0.006*"uci"')]